# CRUD
#### (Create,Read,Update,Delete)
----

## Start
for at bruge denne notebook skal du have powershell core installeret.

Winget:

In [ ]:
winget install Microsoft.PowerShell

Kør koden nedenunder før du kører eksemplerne.

In [ ]:
#!pwsh
Write-Host "Building Project"
dotnet build Efcore.sln | Out-File -FilePath BuildOutput.txt
Write-Host "Project build completed"

#!C#
#r "nuget: Bogus"
#r "nuget: Microsoft.EntityFrameworkCore"
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer"
#r "nuget: Microsoft.EntityFrameworkCore.Sqlite"
#r "DataLayer\bin\Debug\net7.0\DataLayer.dll"

using Microsoft.EntityFrameworkCore;
using DataLayer;

DbContextOptionsBuilder<BlogDbContext> DbOptions = new();

DbOptions.UseSqlite("Data Source=mydb.db;");
//DbOptions.UseSqlServer("<ConnectionString Here>");

using (var db = new BlogDbContext(DbOptions.Options))
{
    Console.WriteLine("Removing old database..");
    db.Database.EnsureDeleted();
    Console.WriteLine("Creating new database..");
    db.Database.EnsureCreated();
}

Console.WriteLine("Seeding Database.");
var users = DbOptions.Options.CreateRandomUsers(10);
var blogs = DbOptions.Options.CreateRandomBlogs(users);
var posts = DbOptions.Options.CreateRandomPosts(10 * 2,blogs,users);
Console.WriteLine("Done.");

// Enable Logging
DbOptions
    .EnableSensitiveDataLogging()
    .LogTo(a => a.Display(),Microsoft.Extensions.Logging.LogLevel.Information);

-----
## Hent fra datbase
Henter data fra databasen hvor teksten skal indeholde "et".

Der der blibver konverteret til lowercase for at få ikke at søge case sensitiv.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Blogs
                    .Where(b => b.BlogName.ToLower().Contains("et"))
                    .ToList();
                    
    Blogs.Display();
}

Her gør vi det samme bare med at vi henter posts med et specifikt Blogid.

In [ ]:
using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Posts
                    .Where(b => b.BlogId == 2)
                    .ToList();
                    
    Blogs.Display();
}

### Data Transformering

Her kan man hente data og tranformere til en anden klasse.

> Observer at entity framework selv laver inner joins for at få data

In [ ]:
class BlogPost {
    public string BlogName { get; set; }
    public string UserName { get; set; }
    public string PostTitle { get; set; }
    public string PostDescription { get; set; }
}

using (var db = new BlogDbContext(DbOptions.Options))
{
    var Blogs = db.Posts
                    .Where(p => p.BlogId == 1)
                    .Select(p => new BlogPost {
                        PostTitle = p.title,
                        PostDescription = p.desctription,
                        UserName = p.User.UserName,
                        BlogName = p.Blog.BlogName
                    }).ToList();
                    
    Blogs.Display();
}